In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_train_model"
table_id_done = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_done"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.data_cleaned` """
data_cleaned = client.query(query).to_dataframe()
data_cleaned.head(1)

,km,origin,body,fuel,name,price,brand,age
0,70000,None,suv,None,Hyundai Santa Fe 2021,780000000,hyundai,4


In [4]:
df = data_cleaned.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   km      5961 non-null   Int64 
 1   origin  5946 non-null   object
 2   body    5663 non-null   object
 3   fuel    4608 non-null   object
 4   name    5961 non-null   object
 5   price   5961 non-null   Int64 
 6   brand   5961 non-null   object
 7   age     5961 non-null   Int64 
dtypes: Int64(3), object(5)
memory usage: 390.2+ KB


In [6]:
df.duplicated().sum()

np.int64(108)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.isna().sum()

km           0
origin      14
body       297
fuel      1325
name         0
price        0
brand        0
age          0
dtype: int64

1.brand

In [9]:
df["brand"].dtype

dtype('O')

In [10]:
df["brand"].unique()

array(['hyundai', 'isuzu', 'jeep', 'kia', 'kenbo', 'lamborghini', 'land',
       'lexus', 'luxgen', 'lynk', 'mazda', 'honda', 'mercedes', 'acura',
       'audi', 'bentley', 'bmw', 'chevrolet', 'citroen', 'daewoo',
       'dongben', 'ferrari', 'ford', 'fuso', 'mg', 'mini', 'mitsubishi',
       'nissan', 'peugeot', 'porsche', 'rolls', 'smart', 'subaru',
       'suzuki', 'thaco', 'toyota', 'vinfast', 'volkswagen', 'volvo',
       'jaguar', 'landrover', 'infiniti', 'ram', 'ssangyong', 'daihatsu',
       'dodge', 'cadillac', 'mercedes benz', 'byd', 'baic', 'fiat', 'gaz',
       'hãng khác', 'sym', 'dongfeng', 'asia', 'rolls royce', 'maybach',
       'lynk&co'], dtype=object)

In [11]:
brand_alias = {
    "mercedes": "mercedes-benz",
    "mercedes benz": "mercedes-benz",

    "rolls": "rolls-royce",
    "rolls royce": "rolls-royce",

    "land": "land rover",
    "landrover": "land rover",

    "lynk": "lynk & co",
    "lynk&co": "lynk & co",
}

In [12]:
df["brand"] = df["brand"].replace(brand_alias)

top_brands = df["brand"].value_counts().nlargest(25).index
df["brand"] = df["brand"].apply(lambda x: x if x in top_brands else "other")


In [13]:
df["brand"].value_counts()

brand
toyota           953
mercedes-benz    722
ford             662
kia              581
hyundai          523
mazda            328
vinfast          305
mitsubishi       296
honda            219
lexus            200
bmw              185
porsche          119
peugeot           97
suzuki            87
chevrolet         78
other             77
nissan            72
land rover        70
mg                66
audi              64
volvo             60
volkswagen        28
daewoo            19
isuzu             16
mini              15
subaru            11
Name: count, dtype: int64

2.origin

In [14]:
df["origin"].unique()

array([None, 'nhập khẩu', 'trong nước'], dtype=object)

In [15]:
df["origin"].value_counts()

origin
nhập khẩu     2984
trong nước    2855
Name: count, dtype: int64

In [16]:
origin_mode_per_brand = df.groupby('brand')['origin'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
origin_mode_per_brand


brand
audi              nhập khẩu
bmw               nhập khẩu
chevrolet         nhập khẩu
daewoo            nhập khẩu
ford              nhập khẩu
honda             nhập khẩu
hyundai          trong nước
isuzu             nhập khẩu
kia              trong nước
land rover        nhập khẩu
lexus             nhập khẩu
mazda            trong nước
mercedes-benz    trong nước
mg                nhập khẩu
mini              nhập khẩu
mitsubishi        nhập khẩu
nissan            nhập khẩu
other             nhập khẩu
peugeot          trong nước
porsche           nhập khẩu
subaru            nhập khẩu
suzuki            nhập khẩu
toyota            nhập khẩu
vinfast          trong nước
volkswagen        nhập khẩu
volvo             nhập khẩu
Name: origin, dtype: object

In [17]:
df['origin'] = df.apply(
    lambda row: origin_mode_per_brand[row['brand']] if pd.isna(row['origin']) else row['origin'],
    axis=1
)


In [18]:
df['origin'].isna().sum()

np.int64(0)

3.body

In [19]:
df["body"].unique()

array(['suv', 'van/minivan', 'hatchback', 'sedan', 'crossover', 'xe tải',
       'mpv', None, 'wagon', 'bán tải', 'convertible', 'coupe',
       'sport car', 'minibus', 'bán tải / pickup', 'truck',
       'convertible/cabriolet', 'coupe (2 cửa)', 'kiểu dáng khác',
       'minivan (mpv)', 'mui trần', 'pick-up (bán tải)',
       'suv / cross over', 'van'], dtype=object)

In [20]:
df['body'] = df['body'].replace({
    'xe tải': 'truck',
    'truck': 'truck',
    'bán tải': 'pickup',
    'bán tải / pickup': 'pickup',
    'pick-up (bán tải)': 'pickup',
    'van/minivan': 'minivan',
    'minivan (mpv)': 'minivan',
    'mpv': 'minivan',
    'suv / cross over': 'suv',
    'crossover': 'suv',
    'convertible/cabriolet': 'convertible',
    'coupe (2 cửa)': 'coupe'
})


In [21]:
body_mode_per_brand = df.groupby('brand')['body'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
body_mode_per_brand

brand
audi                   suv
bmw                    suv
chevrolet            sedan
daewoo               sedan
ford                   suv
honda                  suv
hyundai                suv
isuzu               pickup
kia                    suv
land rover             suv
lexus                  suv
mazda                  suv
mercedes-benz        sedan
mg                   sedan
mini             hatchback
mitsubishi             suv
nissan                 suv
other                  suv
peugeot                suv
porsche                suv
subaru                 suv
suzuki           hatchback
toyota                 suv
vinfast                suv
volkswagen         minivan
volvo                  suv
Name: body, dtype: object

In [22]:
df['body'] = df.apply(
    lambda row: body_mode_per_brand[row['brand']] if pd.isna(row['body']) else row['body'],
    axis=1
)

In [23]:
df['body'].isna().sum()

np.int64(0)

4.fuel

In [24]:
df["fuel"].unique()

array([None, 'Hybrid', 'Điện', 'Xăng', 'Dầu'], dtype=object)

In [25]:
df['fuel'] = df['fuel'].str.lower()

In [26]:
fuel_mode_per_brand_body = df.groupby(["brand","body"])['fuel'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
def fill_fuel(row):
    if pd.isna(row['fuel']):
        return fuel_mode_per_brand_body.get((row['brand'], row['body']), None)
    else:
        return row['fuel']

df['fuel'] = df.apply(fill_fuel, axis=1)

In [27]:
df['fuel'].isna().sum()

np.int64(19)

In [28]:
fuel_mode_per_brand = df.groupby('brand')['fuel'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
df['fuel'] = df.apply(
    lambda row: fuel_mode_per_brand[row['brand']] if pd.isna(row['fuel']) else row['fuel'],
    axis=1
)

df['fuel'].isna().sum()

np.int64(0)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5853 entries, 0 to 5960
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   km      5853 non-null   Int64 
 1   origin  5853 non-null   object
 2   body    5853 non-null   object
 3   fuel    5853 non-null   object
 4   name    5853 non-null   object
 5   price   5853 non-null   Int64 
 6   brand   5853 non-null   object
 7   age     5853 non-null   Int64 
dtypes: Int64(3), object(5)
memory usage: 428.7+ KB


5.outlier

In [30]:
df["price"] = df["price"].astype(float)
df["km"] = df["km"].astype(float)
df["age"] = df["age"].astype(float)

In [31]:
def cap_outliers_group(df, column, group_cols):
    def cap_group(x):
        Q1 = x[column].quantile(0.25)
        Q3 = x[column].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        return x[column].clip(lower, upper)
    
    df[column] = df.groupby(group_cols, group_keys=False).apply(cap_group)
    return df

In [32]:
for col in ['km', 'price', 'age']:
    df = cap_outliers_group(df, col, ['brand', 'body'])

In [33]:
df.head()

,km,origin,body,fuel,name,price,brand,age
0,70000.0,trong nước,suv,xăng,Hyundai Santa Fe 2021,780000000.0,hyundai,4.0
1,78000.0,trong nước,suv,xăng,Hyundai Tucson 2020,685000000.0,hyundai,5.0
2,130000.0,nhập khẩu,minivan,xăng,Hyundai Grand Starex 2 4 MT 2017,555000000.0,hyundai,8.0
3,15000.0,nhập khẩu,minivan,xăng,Hyundai Grand Starex 2 4 MT 2017,450000000.0,hyundai,8.0
4,130000.0,nhập khẩu,minivan,xăng,Hyundai Grand Starex 2 5 MT 2017,555000000.0,hyundai,8.0


In [34]:
df.drop(columns=["name"], inplace=True)

In [35]:
X = df.drop(columns=["price"])
X.head()

,km,origin,body,fuel,brand,age
0,70000.0,trong nước,suv,xăng,hyundai,4.0
1,78000.0,trong nước,suv,xăng,hyundai,5.0
2,130000.0,nhập khẩu,minivan,xăng,hyundai,8.0
3,15000.0,nhập khẩu,minivan,xăng,hyundai,8.0
4,130000.0,nhập khẩu,minivan,xăng,hyundai,8.0


In [36]:
X[X.duplicated(keep=False)]


,km,origin,body,fuel,brand,age
2,130000.0,nhập khẩu,minivan,xăng,hyundai,8.0
4,130000.0,nhập khẩu,minivan,xăng,hyundai,8.0
7,110000.0,nhập khẩu,hatchback,xăng,hyundai,11.0
16,140000.0,nhập khẩu,sedan,xăng,hyundai,8.0
22,140000.0,nhập khẩu,sedan,xăng,hyundai,13.0
...,...,...,...,...,...,...
5888,90000.0,nhập khẩu,suv,xăng,lexus,9.0
5890,70000.0,trong nước,suv,xăng,mitsubishi,7.0
5894,38000.0,trong nước,suv,dầu,hyundai,1.0
5951,77000.0,nhập khẩu,van,dầu,ford,2.0


In [37]:
df_unique = df.groupby(list(X.columns), as_index=False)["price"].mean()
df_unique

,km,origin,body,fuel,brand,age,price
0,101.000,nhập khẩu,suv,xăng,toyota,6.00,1.325000e+09
1,105.000,nhập khẩu,suv,dầu,toyota,8.00,6.270000e+08
2,110.000,nhập khẩu,hatchback,xăng,kia,18.00,1.250000e+08
3,110.000,nhập khẩu,minivan,dầu,kia,10.50,2.000000e+08
4,112.000,nhập khẩu,hatchback,xăng,hyundai,10.00,1.650000e+08
...,...,...,...,...,...,...,...
4808,275000.000,trong nước,van,dầu,mercedes-benz,19.00,9.200000e+07
4809,279765.000,trong nước,minivan,xăng,mercedes-benz,18.00,1.100000e+08
4810,291125.000,nhập khẩu,van,dầu,hyundai,15.00,2.200000e+08
4811,300000.000,trong nước,minivan,dầu,ford,8.00,3.420000e+08


In [38]:
df_unique.duplicated().sum()

np.int64(0)

In [39]:
import json


unique_values = {
    'origin': df['origin'].value_counts().index.tolist(),
    'fuel':   df['fuel'].value_counts().index.tolist(),
    'body':   df['body'].value_counts().index.tolist(),
    'brand':  df['brand'].value_counts().index.tolist(),
}

with open('../../model/unique_values.json', 'w', encoding='utf-8') as f:
    json.dump(unique_values, f, ensure_ascii=False, indent=2)

In [40]:
upload_to_bigquery(df_unique, table_id=table_id_done, if_exists="replace" )

✅ Uploaded 4813 rows to khangtestdbt.xecupredict.data_done


* feature engineering

In [41]:
df_unique["km_group"] = pd.cut(
    df_unique["km"],
    bins=[0, 50000, 100000, 150000, 300000, 1000000],
    labels=["very_low", "low", "medium", "high", "very_high"]
)

In [42]:
def age_risk(age):
    if age <= 3: return "new"
    elif age <= 7: return "mid"
    elif age <= 12: return "old"
    else: return "very_old"

df_unique["age_risk"] = df_unique["age"].apply(age_risk)


In [43]:
df_unique["old_car"] = (df_unique["age"] > 10).map({True: "yes", False: "no"}).astype(object)
df_unique["high_km"] = (df_unique["km"] > 100000).map({True: "yes", False: "no"}).astype(object)


In [44]:
df_unique.head()

,km,origin,body,fuel,brand,age,price,km_group,age_risk,old_car,high_km
0,101.0,nhập khẩu,suv,xăng,toyota,6.0,1.325000e+09,very_low,mid,no,no
1,105.0,nhập khẩu,suv,dầu,toyota,8.0,6.270000e+08,very_low,old,no,no
2,110.0,nhập khẩu,hatchback,xăng,kia,18.0,1.250000e+08,very_low,very_old,yes,no
3,110.0,nhập khẩu,minivan,dầu,kia,10.5,2.000000e+08,very_low,old,yes,no
4,112.0,nhập khẩu,hatchback,xăng,hyundai,10.0,1.650000e+08,very_low,old,no,no


* scale

In [45]:
X = df_unique.copy()

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4813 entries, 0 to 4812
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   km        4813 non-null   float64 
 1   origin    4813 non-null   object  
 2   body      4813 non-null   object  
 3   fuel      4813 non-null   object  
 4   brand     4813 non-null   object  
 5   age       4813 non-null   float64 
 6   price     4813 non-null   float64 
 7   km_group  4813 non-null   category
 8   age_risk  4813 non-null   object  
 9   old_car   4813 non-null   object  
 10  high_km   4813 non-null   object  
dtypes: category(1), float64(3), object(7)
memory usage: 381.1+ KB


In [47]:
X_ = X[["price"]]

In [48]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_cols = [col for col in X.columns 
                    if X[col].dtype == 'object' and col not in X_]
numerical_cols = [col for col in X.columns 
                  if X[col].dtype in ['float64', 'int64', 'Int64'] and col not in X_]

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat = ohe.fit_transform(X[categorical_cols])
cat_feature_names = ohe.get_feature_names_out(categorical_cols)
X_cat_df = pd.DataFrame(X_cat, columns=cat_feature_names, index=X.index)


scaler = StandardScaler()
X_num = scaler.fit_transform(X[numerical_cols])
X_num_df = pd.DataFrame(X_num, columns=numerical_cols, index=X.index)

X_encoded = pd.concat([X_num_df, X_cat_df, X_], axis=1)
X_encoded.shape

(4813, 57)

In [49]:
X_encoded.head()

,km,age,origin_nhập khẩu,origin_trong nước,body_convertible,body_coupe,body_hatchback,body_kiểu dáng khác,body_minibus,body_minivan,...,brand_volvo,age_risk_mid,age_risk_new,age_risk_old,age_risk_very_old,old_car_no,old_car_yes,high_km_no,high_km_yes,price
0,-1.464709,-0.059712,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.325000e+09
1,-1.464619,0.389212,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.270000e+08
2,-1.464507,2.633834,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.250000e+08
3,-1.464507,0.950368,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.000000e+08
4,-1.464462,0.838136,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.650000e+08


In [50]:
from joblib import dump

dump(ohe, "../../model/onehot_encoder.pkl")
dump(scaler,"../../model/scaler.pkl")

['../../model/scaler.pkl']

In [51]:
upload_to_bigquery(X_encoded, table_id=table_id, if_exists="replace" )

✅ Uploaded 4813 rows to khangtestdbt.xecupredict.data_train_model
